## Logistic Regression For Costa Rican Poverty Prediction

### Outline
**1. Project Setup** \
*1.1 Load Data and Packages* \
*1.2 Data Cleaning* \
*1.3 Oversampling* \
\
**2. What is a logistic regression?** \
*2.1 Summary of Approach* \
\
**3. Models** \
*3.1 Basic Models* \
*3.2 Improving Model Performance* \
\
**4. Findings, Limitations and Next Steps**

## 1. Project Setup

**1.1 Load Data**

In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, f1_score, recall_score, classification_report
from collections import Counter
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
current_dir = os.getcwd()
print(os.getcwd())
#parent_dir = os.path.dirname(current_dir)
#os.chdir(parent_dir)
import load_data as ld
df, X_valid, y_valid, train_indices, valid_indices= ld.load_train_data()
from evaluate_classification import evaluate_classification

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score


**1.2 Data Cleaning**

In [ ]:
df, X_valid, y_valid, train_indices, valid_indices= ld.load_train_data()
print(train_indices)

**1.3 Create oversampling:**\
We are looking to better predict Costa Rican Poverty in the following categories: 
- 1 = extreme poverty
- 2 = moderate poverty
- 3 = vulnerable households
- 4 = non vulnerable households

However, as demonstrated in our exploratory analysis, the majority of the data (66%) comes from class 4. Thus, a strong training model is most likely to predict the majority class rather than give accurate weights and biases as to reflect the different categories. 

As such we attempt to do over sampling- with random oversampling, SMOTE and ADASYM. We try random sampling with replacement as well as SVM SMOTE (Support Vector Machine Synthetic Minority Over-sampling Technique), which creates synthetic samples by randomly sampling the characteristics from occurrences in the minority class. The SVM class is designed for small and complex datasets. For SVM SMOTE, a neighborhood is defined using the parameter m_neighbors to decide if a sample is in danger, safe, or noise.

This gives us a number of different datasets to work with, so we can see how oversampling can impact our models.


In [ ]:
X = df.iloc[: , :-1]
y = df.loc[:, 'Target']

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)
train = pd.concat([X_train, y_train], axis=1)
train_df_resampled, train_y_resampled = ld.gen_oversample_data(train, seed = 12)

print("original y train", sorted(Counter(y_train).items()))
print( "Random Sampling", sorted(Counter(train_y_resampled).items()))

X_smote, y_smote = ld.gen_SMOTE_data(train, seed = 12)
print("SMOTE", sorted(Counter(train_y_resampled).items()))


## 2. What is Logistic Regression? 

Logistic Regression is a type of supervised learning algorithm used for classification problems which uses a logistic function to model the probability of a binary or categorical outcome. In our case, our 1-4 levels of poverty produce categorical variables. 

The goal of logistic regression is to find the best parameters of a logistic function that minimizes the difference between the predicted probabilities and the actual outcomes. The logistic function uses the sigmoid function, which maps any real number into a range between 0 and 1, allowing us to interpret the output as a probability. The input features are weighted and combined linearly, and the resulting value is passed through the logistic function to produce a probability. We then use gradient descent to determine what the best classification is for these categories. 

One drawback however, is that it assumes linear relationships which based on the diversity of our data, may not be true in this case. 


### 2.1 What is Regularization
Regularization helps to improve the generalization performance of the model by balancing the bias-variance trade-off and reducing overfitting.  By penalizing the weights of the input features, regularization encourages the model to focus on the most important features that are most relevant to the target variable. Overfitting occurs when a model learns to fit the training data too closely, including noise and irrelevant features, leading to poor generalization performance on new, unseen data. In this case we will try L2 regularization which adds a penalty term proportional to the square of the weights.

## 3. Models
*3.1 Basic Models*


In [ ]:
logreg = LogisticRegression(random_state=42)

# fit the model with data

reg = LogisticRegression(solver='lbfgs', penalty=None, max_iter=1000).fit(X_train, y_train)
y_pred = reg.predict(X_valid)
evaluate_classification(y_pred, y_true = y_valid, l=[1,2,3,4], cm = True)

print("Random Sampling")
reg = LogisticRegression(solver='lbfgs', penalty=None, max_iter=1000).fit(train_df_resampled, train_y_resampled)
y_pred = reg.predict(X_valid)
evaluate_classification(y_pred, y_true = y_valid, l=[1,2,3,4], cm = True)

print("SVM SMOTE")
reg = LogisticRegression(solver='lbfgs', penalty=None, max_iter=1000).fit(X_smote, y_smote)
y_pred = reg.predict(X_valid)
evaluate_classification(y_pred, y_true = y_valid, l=[1,2,3,4], cm = True)


*2.1 With Regularization*

As demonstrated below, regularization is crucial to the success of these models. It also allows us to use a better solver for smaller datasets: liblinear. It also has a substantially higher F1 score. However, the randomly selected do a better job of classifying the data into the correct catergory. 

In [ ]:
logreg = LogisticRegression(random_state=42)

# fit the model with data

reg = LogisticRegression(solver='liblinear').fit(X_train, y_train)
y_pred = reg.predict(X_valid)
evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)

print("Random Sampling")
reg = LogisticRegression(solver='liblinear', penalty='l2').fit(train_df_resampled, train_y_resampled)
y_pred = reg.predict(X_valid)
evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)

print("SVM SMOTE")
reg = LogisticRegression(solver='liblinear', penalty='l2').fit(X_smote, y_smote)
y_pred = reg.predict(X_valid)
evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)


*3.2 Improve Model Performance*

Adding Feature selection to these models as shown below does not seem to increase accuracy in a meaningful way - Particularly in characterization of 1's and two's. 

In [ ]:
def Run_models():
    print("Random forest")
    reg = RandomForestClassifier()
    reg.fit(X_smote, y_smote)
    y_pred = reg.predict(X_valid)
    evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)

    print("Random forest -RFE")
    reg = RandomForestClassifier()
    reg.fit(train_df_resampled, train_y_resampled)
    y_pred = reg.predict(X_valid)
    evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)


    print("With Smote- RFE")
    #Recursive feature elimination
    rfe = RFE(estimator=LogisticRegression(solver='liblinear', penalty='l2'),n_features_to_select = 6, step = 1)
    rfe.fit(X_smote, y_smote)
    y_pred = reg.predict(X_valid)
    evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)

    print("Features selected:With Smote- RFE")
    print(X_train.iloc[:,[64,138,126,127,107,108,6]].columns)

    print("With Random Selection- RFE")
    #Recursive feature elimination
    rfe = RFE(estimator=LogisticRegression(solver='liblinear', penalty='l2'),n_features_to_select = 6, step = 1)
    rfe.fit(train_df_resampled, train_y_resampled)
    y_pred = reg.predict(X_valid)
    evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)
    print("Features selected:With Random Selection- RFE")
    print(X_train.iloc[:,[36,37,42,45,107,108]].columns)


    print("With Smote- Variance Threshold")
    # get rid of features without a lot of variance 
    from sklearn.feature_selection import VarianceThreshold
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    a= X_smote.copy()
    rev_x = sel.fit_transform(a)
    reg = LogisticRegression(solver='liblinear', penalty='l2').fit(rev_x, y_smote)

    y_pred = reg.predict(sel.transform(X_valid.copy()))
    evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)

    print("With Random Selection-Variance threshold")

    # get rid of features without a lot of variance 
    from sklearn.feature_selection import VarianceThreshold
    sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
    rev_x = sel.fit_transform(train_df_resampled.copy())
    reg = LogisticRegression(solver='liblinear', penalty='l2').fit(rev_x, train_y_resampled)
    a= X_valid.copy()
    y_pred = reg.predict(sel.transform(a))
    evaluate_classification(y_pred, y_true = y_valid, labels=[1,2,3,4], cm = True)
Run_models()

In [ ]:
reg = LogisticRegression(solver='liblinear', penalty='l2')

bagging_classifier_rf = BaggingClassifier(
    base_estimator=reg,
    n_estimators=10,
    random_state=42)

bagging_classifier_rf.fit(X_smote, y_smote)
bc_rf_predictions = bagging_classifier_rf.predict(X_valid)
evaluate_classification(bc_rf_predictions, y_valid, l=[1,2,3,4], cm=True, return_vals=False)

In [ ]:
from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
rev_x = sel.fit_transform(X_smote.copy())
reg = RandomForestClassifier(random_state = 12,
                        n_estimators = 1600,
                        min_samples_split = 2,
                        min_samples_leaf = 1,
                        max_features = 'sqrt',
                        max_depth = 100,
                        bootstrap = False).fit(rev_x, y_smote)
a= X_valid.copy()
y_pred = reg.predict(sel.transform(a))
evaluate_classification(y_pred, y_true = y_valid, l=[1,2,3,4], cm = True)

In [ ]:

from sklearn.feature_selection import VarianceThreshold
sel = VarianceThreshold(threshold=(.8 * (1 - .8)))
a= X_smote.copy()
rev_x = sel.fit_transform(a)
reg = LogisticRegression(solver='liblinear', penalty='l2').fit(rev_x, y_smote)

bagging_classifier_rf = BaggingClassifier(
    base_estimator=reg,
    n_estimators=10,
    random_state=42)

bagging_classifier_rf.fit(rev_x, y_smote)
bc_rf_predictions = bagging_classifier_rf.predict(sel.transform(X_valid.copy()))
evaluate_classification(bc_rf_predictions, y_valid, l=[1,2,3,4], cm=True, return_vals=False)

**4. Findings, Limitations and Next Steps**

Our models regardless of oversampling are fairly accurate at predicting 4, but need more data to provide any meaningful accuracy using logistic regression to predict the lower categories at any meaningful amount of accuracy. While, if we reference the logistic regression function, without overweighting we only classify the majority class 4. With overweighting and feature selection methods, we get our best predictors using regularized logistic regression in terms of classification accuracy and F1 score. But given the diversity of the results, and column selection in model development, it is clear feature selection as opposed to logistic regression usage is the most important part of this work. It is unclear if regularized logistic regression will be the best indicator going forward.

